# Urban Tree Canopy - Part 2 (UTC Processing)

## After completing Part 1 (LiDAR Processing or Raster Classification) AND the output has been QA/QC'd, this script will process the polygons into UTC and C2C shapefiles that are ready to be uploaded. The pair of shapefiles will be in the cities respective folder for labeled as their FIPS code, and the TIFs will be stand alone. Edit the Cities_All layer from Fernando to reflect where the data is from and what year. Give to Fernando once complete. 

## BEFORE YOU BEGIN
1.) Ensure the output for Part 1 has been uploaded to a new GDB in your folders. <br>
2.) Change the input in the first cell to reflect where the newly created GDB from Part 1 is, and where you want the final outputs to be saved.<br>


In [ ]:
import os
import time
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False
arcpy.env.parallelProcessingFactor = "80%"  # Use all available cores

#CHANGE THESE
final_gdb = r'D:\ArcGIS_Projects\UTC\UTC\UTC_Final.gdb'
arcpy.env.workspace = r'D:\ArcGIS_Projects\UTC\UTC\Testing.gdb'
scratch_gdb = r"D:\ArcGIS_Projects\UTC\UTC\scratch.gdb"

etj = r"D:\ArcGIS_Projects\UTC\UTC\UTC.gdb\ETJ"
city_boundary = r"D:\ArcGIS_Projects\UTC\UTC\UTC.gdb\City_Boundary"
chhd = r"D:\ArcGIS_Projects\UTC\UTC\UTC.gdb\CHHD"

#DON'T CHANGE THESE
feature_classes = arcpy.ListFeatureClasses()
cities = []

for fc in feature_classes:
    cities.append(fc)
print(cities)

In [ ]:
for city in cities:
    print(f"Starting {city} Processing")
    with arcpy.da.SearchCursor(etj, ['NAME10', 'PlaceFIPS'], f"NAME10 = '{city}'") as cursor:
        fips_code = next(cursor)[1]
    print(f"FIPS Code: {fips_code}")
    output_folder = os.path.join(r"D:\ArcGIS_Projects\UTC\Output", f"{fips_code}")
    os.makedirs(output_folder, exist_ok=True)
    # Temporary output paths in the scratch workspace
    city_dissolve = f"in_memory\\{city}_Dissolve"
    chhd_clip = f"in_memory\\{city}_CHHD"
    chhd_canopy_clip = f"in_memory\\{city}_CHHD_canopy_clip"
    city_etj_temp = f"in_memory\\{city}_ETJ_TEMP"
    city_clip = f"in_memory\\{city}_City"
    city_boundary_clip = f"in_memory\\{city}_Boundary"
    city_etj = f"in_memory\\{city}_ETJ"
    city_utc = f"in_memory\\{city}_utc"
    city_c2c_temp = f"in_memory\\{city}_c2c_temp"
    city_c2c = f"in_memory\\{city}_c2c"

    arcpy.conversion.ExportFeatures(etj, city_etj, f"NAME10 = '{city}'")
    arcpy.conversion.ExportFeatures(city_boundary, city_boundary_clip, f"NAME10 = '{city}'")
    
    # Dissolve the city polygons
    arcpy.management.Dissolve(city, city_dissolve)
    
    arcpy.analysis.Clip(chhd, city_etj, chhd_clip)
    arcpy.analysis.Clip(city_dissolve, city_etj, city_etj_temp)
    
    # Check if the clip output is empty
    result = arcpy.GetCount_management(chhd_clip)
    feature_count = int(result.getOutput(0))  # Get the number of features

    if feature_count == 0:
        print(f"No CHHD data for {city}. Skipping related steps.\n")
    else:
        print(f"CHHD data found for {city}. Proceeding with further analysis.")
    
    # Clip dissolved city by city boundary
        arcpy.analysis.Clip(city_dissolve, city_boundary_clip, city_clip)

        # Erase the portion of city within ETJ
        arcpy.analysis.Erase(city_etj_temp, city_clip, city_etj)

        # Add fields to the city and ETJ layers
        fields = [['Name', 'TEXT'], ['CL_ET', 'SHORT'], ['Area', 'FLOAT'], ['Name_Join', 'TEXT']]
        arcpy.management.AddFields(city_clip, fields)
        arcpy.management.AddFields(city_etj, fields)

        # Calculate fields for ETJ
        arcpy.management.CalculateField(city_etj, "Name", f"'{city}_ETJ'", 'PYTHON3')
        arcpy.management.CalculateField(city_etj, "Name_Join", f"'{city}'", 'PYTHON3')
        arcpy.management.CalculateField(city_etj, "CL_ET", -1, 'PYTHON3')
        arcpy.management.CalculateGeometryAttributes(city_etj, [['Area', 'AREA_GEODESIC']], "METERS", "ACRES_US")
        arcpy.management.JoinField(city_etj, 'Name_Join', etj, 'NAME10', 'PlaceFIPS')


        # Calculate fields for City boundary
        arcpy.management.CalculateField(city_clip, "Name", f"'{city}'", 'PYTHON3')
        arcpy.management.CalculateField(city_clip, "Name_Join", f"'{city}'", 'PYTHON3')
        arcpy.management.CalculateField(city_clip, "CL_ET", 0, 'PYTHON3')
        arcpy.management.CalculateGeometryAttributes(city_clip, [['Area', 'AREA_GEODESIC']], "METERS", "ACRES_US")
        arcpy.management.JoinField(city_clip, 'Name_Join', city_boundary, 'NAME10', 'PlaceFIPS')


        # Merge city and ETJ layers

        arcpy.management.Merge([city_clip, city_etj], city_utc)

        # Add and calculate C2C field
        arcpy.management.AddField(city_utc, 'C2C', 'TEXT')
        arcpy.management.CalculateField(city_utc, 'C2C', "'At risk'", 'PYTHON3')

        # Erase the portion of CHHD that intersects with the city UTC layer
        arcpy.analysis.Erase(chhd_clip, city_utc, city_c2c_temp)

        # Add fields to c2c_temp layer
        arcpy.management.AddFields(city_c2c_temp, fields)

        # Calculate fields for c2c_temp
        arcpy.management.CalculateField(city_c2c_temp, "Name", f"'{city}'", 'PYTHON3')
        arcpy.management.CalculateField(city_c2c_temp, "Name_Join", f"'{city}'", 'PYTHON3')
        arcpy.management.CalculateField(city_c2c_temp, "CL_ET", 2, 'PYTHON3')
        arcpy.management.CalculateGeometryAttributes(city_c2c_temp, [['Area', 'AREA_GEODESIC']], "METERS", "ACRES_US")
        arcpy.management.JoinField(city_c2c_temp, 'Name_Join', etj, 'NAME10', 'PlaceFIPS')
        arcpy.management.DeleteField(city_c2c_temp, 'GRIDCODE')

        # Add and calculate C2C field for c2c_temp
        arcpy.management.AddField(city_c2c_temp, 'C2C', 'TEXT')
        arcpy.management.CalculateField(city_c2c_temp, 'C2C', "'To grow'", 'PYTHON3')

        # Merge final layers
        arcpy.analysis.Clip(city_utc, chhd_clip, chhd_canopy_clip)
        arcpy.management.Merge([chhd_canopy_clip, city_c2c_temp], city_c2c)

        # Copy final results to the output geodatabase
        output_utc = os.path.join(final_gdb, f'{city}_utc')
        arcpy.management.CopyFeatures(city_utc, output_utc)
        arcpy.management.DeleteField(output_utc, 'Name_Join')
        arcpy.management.DeleteField(output_utc, "C2C")

        output_c2c = os.path.join(final_gdb, f'{city}_c2c')
        arcpy.management.CopyFeatures(city_c2c, output_c2c)
        arcpy.management.DeleteField(output_c2c, 'Name_Join')

        utc_shapefile = os.path.join(output_folder, f"{city}_utc.shp")
        c2c_shapefile = os.path.join(output_folder, f"{city}_c2c.shp")
        arcpy.conversion.FeatureClassToShapefile([output_utc], output_folder)
        arcpy.conversion.FeatureClassToShapefile([output_c2c], output_folder)

        utc_tif = os.path.join(r'D:\ArcGIS_Projects\UTC\Output', f"{city}_utc.tif")
        c2c_tif = os.path.join(r'D:\ArcGIS_Projects\UTC\Output', f"{city}_c2c.tif")

        cell_size = 1  # Set cell size in the desired units (e.g., meters)


        # Convert feature classes to raster
        arcpy.conversion.PolygonToRaster(utc_shapefile, 'CL_ET', utc_tif, cell_assignment="MAXIMUM_AREA", priority_field=None, cellsize=cell_size)

        arcpy.conversion.PolygonToRaster(c2c_shapefile, 'CL_ET', c2c_tif, cell_assignment="MAXIMUM_AREA", priority_field=None, cellsize=cell_size)

        arcpy.management.Delete(city_dissolve)
        arcpy.management.Delete(chhd_clip)
        arcpy.management.Delete(chhd_canopy_clip)
        arcpy.management.Delete(city_etj_temp)
        arcpy.management.Delete(city_clip)
        arcpy.management.Delete(city_boundary_clip)
        arcpy.management.Delete(city_etj)
        arcpy.management.Delete(city_utc)
        arcpy.management.Delete(city_c2c_temp)
        arcpy.management.Delete(city_c2c)

        print(f'Analysis completed for {city}\n')

print("UTC Processing Complete")